A feature of MotionClouds is the ability to precisely tune the precision of information  following the principal axes. One which is particularly relevant for the primary visual cortical area of primates (area V1) is to tune the otirentation mean and bandwidth.

## Studying the role of orientation bvandwidth in V1 using MotionClouds

<!-- TEASER_END -->

This is part of a larger study to tune [orientation bandwidth](http://blog.invibe.net/categories/orientation.html).



In [1]:
import os
import numpy as np
import MotionClouds as mc
downscale = 1
fx, fy, ft = mc.get_grids(mc.N_X//downscale, mc.N_Y//downscale, mc.N_frame//downscale)

name = '2019-01-09_protocol'
mc.figpath = os.path.join('output/', name)
if not(os.path.isdir(mc.figpath)): os.mkdir(mc.figpath)

In [2]:
N_X = fx.shape[0]
width = 29.7*256/1050

sf_0 = 4.*width/N_X
B_V = .0     # BW temporal frequency (speed plane thickness)
B_sf = sf_0   # BW spatial frequency
theta = 0.0   # Central orientation
B_thetas = [np.pi/32, np.pi/16, np.pi/8, np.pi/4]
seed = 12234565

In [3]:
B_theta_low, B_theta_high = np.min(B_thetas), np.max(B_thetas)
mc1 = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0., B_V=B_V, sf_0=sf_0, B_sf=B_sf, theta=theta, B_theta=B_theta_low)
mc2 = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0., B_V=B_V, sf_0=sf_0, B_sf=B_sf, theta=theta, B_theta=B_theta_high)
name_ = name + '_narrow'
mc.figures(mc1, name_, seed=seed, figpath=mc.figpath)
mc.in_show_video(name_, figpath=mc.figpath)
name_ = name + '_broad'
mc.figures(mc2, name_, seed=seed, figpath=mc.figpath)
mc.in_show_video(name_, figpath=mc.figpath)

/usr/local/lib/python3.7/site-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


## designing one block with different orientations

The [Exponential distribution](https://en.wikipedia.org/wiki/Exponential_distribution) is the probability distribution that describes the time between events in a Poisson point process

In [3]:
import os
def make_one_block(N_X, N_Y, seed, B_thetas, N_frame_total,  
                   N_frame_mean=6, N_theta=12, contrast=1.):
    fx, fy, ft = mc.get_grids(N_X, N_Y, 1)

    rng = np.random.RandomState(seed)

    N_frame = 0
    im = np.zeros((N_X, N_Y, 0))
    disk = mc.frequency_radius(fx, fy, ft) < .5
    
    while N_frame < N_frame_total:
        N_frame_sub = int(rng.exponential(N_frame_mean))
        theta = np.int(rng.rand()*N_theta) * np.pi / N_theta
        B_theta = B_thetas[rng.randint(len(B_thetas))]
        mc_i = mc.envelope_gabor(fx, fy, ft, 
                                         V_X=0., V_Y=0., B_V=0., 
                                         sf_0=sf_0, B_sf=B_sf, 
                                         theta=theta, B_theta=B_theta)
        im_ = np.zeros((N_X, N_Y, 1))
        im_ += mc.rectif(mc.random_cloud(mc_i, seed=seed+N_frame), contrast=contrast)
        im_ *= disk # masking outside the disk 
        im_ += .5*(1-disk) # gray outside the disk
        im_ = im_ * np.ones((1, 1, N_frame_sub)) #  expand to N_frame_sub frames
        im_[0, 0, :] = 0. # black dot on the top left
        im_[0, 0, 0] = 1. # white dot on the top left at time of switch
        im = np.concatenate((im, im_), axis=-1) # montage
        N_frame = im.shape[-1]
    return im[:, :, :N_frame_total]

N_X, N_Y = mc.N_X, mc.N_Y
im = make_one_block(N_X, N_Y, seed=1234, B_thetas=B_thetas, 
                    N_frame_total=200, N_frame_mean=12, N_theta=12)
name_ = name + '_block'
mc.anim_save(im, os.path.join(mc.figpath, name_), figpath=mc.figpath)
mc.in_show_video(name_, figpath=mc.figpath)

""


In [5]:
# TODO: show that using the same seed always returns the same movie

## summary of the protocol

We show successive blocks of X seconds consisting on average of sub-blocks of 1000ms.

- orientation and bandwidths is changed within one block

To summarize this (POST = TO BE DONE AT THE PRESENTATION SOFTWARE LEVEL):

* Rapid presentation of 20 stimuli within a circular disk (POST) during 100 ms @ 60Hz = 6 frames each
* fixed parameters:
 - mean spatial frequency tuned for optimal neural tuning,
 - frequency bandwidth tuned for optimal neural tuning (0.1 - 5 cyc/deg),
 - temporal frequency bandwidth tuned for optimal neural tuning (1-15Hz (singh et al + Henriksson et al)) / one bandwidth in speed: dynamic (B_V=.5) or static (my preference for this short period)

* parameters:
 - 12 orientations including cardinals
 - 4 orientation bandwidths (pi/4, pi/8, pi/16, pi/32),
 - 3 different seeds: 42, 1973 and 1996 (completely arbitrary)
 - 4 different contrasts 0.03 0.07 0.18 0.42 (Boynton et al) (POST)

Grand total for one block is (12 orientations times 6 BWo + 1) * 2s :

In [6]:
print('One super-block=', (7*3), ' conditions')
print('One super-block=', (7*3) * 2, ' seconds')
print('16 repetitions of one super-block=', (7*3) * 16, ' conditions')
print('16 repetitions of one super-block=', (7*3) * 2 * 16, ' seconds')

One super-block= 21  conditions
One super-block= 42  seconds
16 repetitions of one super-block= 336  conditions
16 repetitions of one super-block= 672  seconds


One session amounts  $672$ seconds, that is about $10$ minutes. 

Let's first get the optimal values for the
 - mean spatial frequency tuned for optimal neural tuning,
 - spatial frequency bandwidth tuned for optimal neural tuning

In [7]:
viewingDistance = 57 # cm # TODO = me donner ces informations!
screen_width_cm = 33 # cm # TODO = me donner ces informations!
#un deg / cm
print('visual angle of the screen', 2*np.arctan(screen_width_cm/2/viewingDistance)*180/np.pi)
print('degrees per centimeter', 2*np.arctan(screen_width_cm/2/viewingDistance)*180/np.pi/screen_width_cm)

visual angle of the screen 32.28867756056697
degrees per centimeter 0.9784447745626353


In [8]:
screen_width_px = 1024 # pixels 
screen_height_px = 768 # pixels

#un pixel = 33/800 deg
deg_per_px = 2*np.arctan(screen_width_cm/2/viewingDistance)*180/np.pi/screen_width_px
print('degrees per pixel', deg_per_px)

degrees per pixel 0.03153191168024118


The central spatial frequency ``sf_0`` is defined as the frequency (number of cycles) *per pixel*, so that to get 

In [9]:
print('width of these motion clouds (', mc.N_X, ', ', mc.N_Y, ')')
print('width of stimulus in degrees', mc.N_X * deg_per_px)
phi_sf_0 = 2. # Optimal spatial frequency [cpd]
print('Optimal spatial frequency in cycles per degree', phi_sf_0)
print('Optimal spatial frequency in cycles per window = ', phi_sf_0 *  mc.N_X * deg_per_px)
sf_0 = phi_sf_0 * deg_per_px
print('cycles per pixel = ', sf_0)

width of these motion clouds ( 256 ,  256 )
width of stimulus in degrees 8.072169390141742
Optimal spatial frequency in cycles per degree 2.0
Optimal spatial frequency in cycles per window =  16.144338780283483
cycles per pixel =  0.06306382336048236


Similarly the spatial frequeny bandwidth as a function of the experimental parameters:

In [10]:
phi_sf_0 = 2. # Optimal spatial frequency [cpd]
phi_B_sf = 2. # Optimal spatial frequency bandwidth [in octaves]
B_Sf = sf_0 # good qualitative approximation

In [11]:
phi_B_V = 5. # Optimal temporal frequency bandwidth [Hz]

#tf_opt = 1 # Hz
T = 0.250            # Stimulus duration [s] 
framerate = 100.    # Refreshing rate in [Hz]
Bv = phi_B_V # good qualitative approximation 

In one script:

In [12]:
!rm -fr output/{name}/*seed*

In [13]:
import numpy as np
import MotionClouds as mc
import os


name = '2019-01-09_protocol'
mc.figpath = os.path.join('output/', name)
if not(os.path.isdir(mc.figpath)): os.mkdir(mc.figpath)
vext = '.png'
vext = '.mp4'

# Experimental constants 
contrast = 1.
# Clouds parameters in absolute units
N_X = 512
width = 29.7*N_X/1050
phi_sf_0 = 2. # Optimal spatial frequency [cpd]

sf_0 = phi_sf_0*width/N_X
B_sf = sf_0   # BW spatial frequency
B_V = .5     # BW temporal frequency (speed plane thickness) WARNING temporal autocorrelation depends on N_frame

# generate zip files
dry_run = True
dry_run = False
      
for seed in [2016 + i for i in range(7)]:
    name_ = name + '_seed_' + str(seed)
    if not dry_run:
        if  not(os.path.isfile(os.path.join(mc.figpath, name_ + vext))):
            im = make_one_block(N_X, N_X, seed=seed, B_thetas=B_thetas, N_frame_total=200, N_frame_mean=25, N_theta=12)
            mc.anim_save(mc.rectif(im, contrast=contrast), os.path.join(mc.figpath, name_), vext=vext)
        else:
            print(' MC ' + os.path.join(mc.figpath, name_) + ' already done')
    else:
        print(' MC ' + os.path.join(mc.figpath, name_) + ' skipped  (dry run)')


## some book keeping for the notebook

In [14]:
%load_ext version_information
%version_information numpy, scipy, matplotlib, MotionClouds

Software versions
Python 3.7.2 64bit [Clang 10.0.0 (clang-1000.11.45.5)]
IPython 7.2.0
OS Darwin 18.2.0 x86_64 i386 64bit
numpy 1.15.4
scipy 1.2.0
matplotlib 3.0.2
MotionClouds 20180606
Wed Jan 09 16:33:18 2019 CET